# Emergency Vehicle Route Testing

This notebook tests the pathfinding system for emergency vehicles.

In [5]:
import sys
sys.path.append('..')

from utils.pathfinding import PathFinder, Location
import numpy as np
import folium
from IPython.display import display
import random
import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

## 1. Initialize PathFinder

In [6]:
# Initialize pathfinder
pathfinder = PathFinder()

# Load city graph (using Bangalore as example)
pathfinder.load_area_graph("Bangalore, India")

# Add sample emergency locations
hospitals = [
    Location("Victoria Hospital", 12.9634, 77.5855, "hospital"),
    Location("Fortis Hospital", 12.9561, 77.5921, "hospital"),
    Location("Apollo Hospital", 12.9784, 77.5733, "hospital"),
    Location("Manipal Hospital", 12.9583, 77.6457, "hospital")
]

fire_stations = [
    Location("Central Fire Station", 12.9757, 77.5921, "fire_station"),
    Location("HSR Fire Station", 12.9141, 77.6446, "fire_station"),
    Location("Yelahanka Fire Station", 13.0994, 77.5900, "fire_station")
]

police_stations = [
    Location("Central Police Station", 12.9716, 77.5946, "police_station"),
    Location("Koramangala Police Station", 12.9279, 77.6271, "police_station"),
    Location("Whitefield Police Station", 12.9698, 77.7500, "police_station")
]

# Register locations
for location in hospitals + fire_stations + police_stations:
    pathfinder.add_emergency_location(location)

## 2. Test Basic Route Finding

In [7]:
def test_route(current_lat, current_lon, vehicle_type):
    """Test route finding for a specific scenario"""
    print(f"Finding route for {vehicle_type} at ({current_lat}, {current_lon})")
    
    try:
        route_coords, destination = pathfinder.find_optimal_route(
            current_lat,
            current_lon,
            vehicle_type
        )
        
        print(f"\nDestination found: {destination.name}")
        print(f"Destination coordinates: ({destination.lat}, {destination.lon})")
        print(f"Route length: {len(route_coords)} points")
        
        # Visualize route
        map_vis = pathfinder.visualize_route(route_coords, destination)
        display(map_vis)
        
        return route_coords, destination
        
    except Exception as e:
        print(f"Error: {e}")
        return None, None

# Test scenarios
test_cases = [
    (12.9716, 77.5946, "ambulance"),  # City center to hospital
    (12.9279, 77.6271, "fire_engine"), # Koramangala to fire incident
    (12.9698, 77.7500, "police")       # Whitefield to police station
]

for lat, lon, vehicle in test_cases:
    print("\n" + "="*50)
    test_route(lat, lon, vehicle)


Finding route for ambulance at (12.9716, 77.5946)
Error: not enough values to unpack (expected 3, got 2)

Finding route for fire_engine at (12.9279, 77.6271)
Error: not enough values to unpack (expected 3, got 2)

Finding route for police at (12.9698, 77.75)
Error: not enough values to unpack (expected 3, got 2)


## 3. Test with Traffic Weights

In [ ]:
def generate_traffic_weights(num_edges=100):
    """Generate random traffic weights for testing"""
    traffic_weights = {}
    for i in range(num_edges):
        # Random edge ID and weight (1.0 = normal, >1.0 = congested)
        edge_id = i
        weight = random.uniform(1.0, 3.0)
        traffic_weights[edge_id] = weight
    return traffic_weights

def compare_routes(current_lat, current_lon, vehicle_type):
    """Compare routes with and without traffic considerations"""
    # Generate random traffic
    traffic_weights = generate_traffic_weights()
    
    print("Testing route without traffic:")
    result1 = test_route(current_lat, current_lon, vehicle_type)
    if result1[0] is None:
        print("Could not find route without traffic")
        return
    route1, dest1 = result1
    
    print("\nTesting route with traffic:")
    try:
        route2, dest2 = pathfinder.find_optimal_route(
            current_lat,
            current_lon,
            vehicle_type,
            traffic_weights=traffic_weights
        )
    except Exception as e:
        print(f"Could not find route with traffic: {e}")
        return
    
    # Visualize both routes on same map
    center_lat = sum(lat for lat, _ in route1) / len(route1)
    center_lon = sum(lon for _, lon in route1) / len(route1)
    m = folium.Map(location=[center_lat, center_lon], zoom_start=13)
    
    # Draw original route
    folium.PolyLine(
        route1,
        weight=2,
        color='blue',
        opacity=0.8,
        popup='Without traffic'
    ).add_to(m)
    
    # Draw traffic-aware route
    folium.PolyLine(
        route2,
        weight=2,
        color='red',
        opacity=0.8,
        popup='With traffic'
    ).add_to(m)
    
    # Add markers
    folium.Marker(
        [current_lat, current_lon],
        popup='Start',
        icon=folium.Icon(color='green')
    ).add_to(m)
    
    folium.Marker(
        [dest1.lat, dest1.lon],
        popup=f'{dest1.name}\n(without traffic)',
        icon=folium.Icon(color='blue')
    ).add_to(m)
    
    if dest1 != dest2:
        folium.Marker(
            [dest2.lat, dest2.lon],
            popup=f'{dest2.name}\n(with traffic)',
            icon=folium.Icon(color='red')
        ).add_to(m)
    
    display(m)

# Test with traffic
compare_routes(12.9716, 77.5946, "ambulance")

Testing route without traffic:
Finding route for ambulance at (12.9716, 77.5946)
Error: not enough values to unpack (expected 3, got 2)

Testing route with traffic:


ValueError: not enough values to unpack (expected 3, got 2)

## 4. Performance Testing

In [ ]:
def test_pathfinding_performance(num_tests=100):
    """Test pathfinding performance with random start points"""
    # Bangalore bounding box
    lat_range = (12.8, 13.1)
    lon_range = (77.4, 77.8)
    
    vehicle_types = ['ambulance', 'fire_engine', 'police']
    computation_times = []
    route_lengths = []
    success_count = 0
    
    for _ in tqdm(range(num_tests), desc='Testing routes'):
        # Generate random start point
        start_lat = random.uniform(*lat_range)
        start_lon = random.uniform(*lon_range)
        vehicle_type = random.choice(vehicle_types)
        
        try:
            # Time the route computation
            start_time = time.time()
            route, dest = pathfinder.find_optimal_route(
                start_lat,
                start_lon,
                vehicle_type
            )
            compute_time = time.time() - start_time
            
            computation_times.append(compute_time)
            route_lengths.append(len(route))
            success_count += 1
            
        except Exception as e:
            print(f"Failed route: {e}")
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    ax1.hist(computation_times, bins=20)
    ax1.set_title('Computation Time Distribution')
    ax1.set_xlabel('Time (seconds)')
    ax1.set_ylabel('Count')
    
    ax2.hist(route_lengths, bins=20)
    ax2.set_title('Route Length Distribution')
    ax2.set_xlabel('Number of Points')
    ax2.set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nPerformance Summary:")
    print(f"Success rate: {success_count/num_tests*100:.1f}%")
    print(f"Average computation time: {np.mean(computation_times):.3f}s")
    print(f"Average route length: {np.mean(route_lengths):.1f} points")

# Run performance tests
test_pathfinding_performance()